In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..
%pwd

/home/markus/Documents/regression-prediction


'/home/markus/Documents/regression-prediction'

In [3]:
from experiments.modeleval_utils import *

/home/markus/Documents/regression-prediction/venv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [11]:
import pickle
output_dir = 'experiments/results_FS'
with  open(os.path.join(output_dir, 'evaluations.pickle'), 'rb') as f:
    evaluations_by_config = pickle.load(f)

/home/markus/Documents/regression-prediction/venv/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [22]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [37]:
scoring = 'average_precision'
all_sampler_ranks = []
for output_dir in ['experiments/results', 'experiments/results_FS']:
    for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance')]: #('bugbug_buglevel', 'regression')
        for feature_type in ['traditional', 'bow']:
            if output_dir == 'experiments/results_FS' and feature_type == 'bow':
                continue
            for model in models:
                try:
                    results = get_results(output_dir, data, feature_type, target, scoring, model)
                    results = results.replace({
                        None: 'No sampling',
                        'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                        'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                        'SMOTE(random_state=0)': 'SMOTE'
                        })
                    sampler_ranks = results.groupby('param_sampler')[['mean_test_score']].max().sort_values('mean_test_score', ascending=False)
                    sampler_ranks['rank'] = np.arange(1,5)
                    sampler_ranks['model'] = np.full(4, model)
                    all_sampler_ranks.append(sampler_ranks)
                except FileNotFoundError:
                    pass
all_sampler_ranks = pd.concat(all_sampler_ranks)
all_sampler_ranks

,mean_test_score,rank,model
param_sampler,,,
No sampling,0.0304,1,lr
Random Over-Sampling,0.0294,2,lr
SMOTE,0.0287,3,lr
Random Under-Sampling,0.0270,4,lr
No sampling,0.0365,1,svm
...,...,...,...
SMOTE,0.0863,4,xgb
Random Over-Sampling,0.1040,1,mlp
SMOTE,0.0967,2,mlp


In [38]:
t = all_sampler_ranks.groupby('param_sampler')[['rank']].mean().sort_values('rank', ascending=True)
t.index.name = 'Sampling Method'
t = t.rename({'rank': 'Average Rank'}, axis=1)
t

,Average Rank
Sampling Method,
Random Over-Sampling,2.0357
No sampling,2.1786
SMOTE,2.8214
Random Under-Sampling,2.9643


In [25]:
print(t.to_latex())

\begin{tabular}{lr}
\toprule
{} &  Average Rank \\
Sampling Method       &               \\
\midrule
Random Over-Sampling  &        1.9444 \\
No sampling           &        2.2778 \\
Random Under-Sampling &        2.7222 \\
SMOTE                 &        3.0556 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_14058/3674067139.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(t.to_latex())


In [35]:
scoring = 'average_precision'
best_sampler = []
for (data, target) in [('fixed_defect_szz', 'performance'), ('bugbug_buglevel', 'performance'), ('bugbug_buglevel', 'regression')]:
    for feature_type in ['traditional', 'bow']:
        for model in models:
            try:
                results = get_results(output_dir, data, feature_type, target, scoring, model)
                results = results.replace({
                    None: 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })
                best_sampler.append({
                    'Data': data,
                    'Feature Type': feature_type,
                    'Target': target,
                    'Model': model_names[model],
                    'Sampling':results.iloc[0]['param_sampler'],
                    'AP (val.)': results.iloc[0]['mean_test_score']
                    })
            except FileNotFoundError:
                pass

best_sampler = pd.DataFrame(best_sampler)
best_sampler = best_sampler.replace({
    'fixed_defect_szz': 'SZZ',
    'bugbug_buglevel': 'bugbug'
})
best_sampler = best_sampler.replace({
    'performance': 'perf. regressions',
    'regression': 'all regressions',
    'bow': 'bag-of-words'
})
best_sampler.set_index(pd.MultiIndex.from_frame(best_sampler[['Data', 'Target', 'Feature Type']]), inplace=True)
best_sampler = best_sampler.drop(['Data', 'Target', 'Feature Type'], axis=1)
best_sampler

Model        Sampling  \
Data   Target            Feature Type                                           
SZZ    perf. regressions traditional      Logistic Regression     No sampling   
                         traditional   Support Vector Machine     No sampling   
                         traditional            Random Forest   Over-Sampling   
                         traditional                  XGBoost   Over-Sampling   
                         traditional   Multi-Layer Perceptron     No sampling   
                         bag-of-words     Logistic Regression     No sampling   
                         bag-of-words  Support Vector Machine   Over-Sampling   
                         bag-of-words           Random Forest     No sampling   
                         bag-of-words                 XGBoost     No sampling   
bugbug perf. regressions traditional      Logistic Regression           SMOTE   
                         traditional   Support Vector Machine           SMOTE   
                         traditional            Random Forest   Over-Sampling   
                         traditional                  XGBoost     No sampling   
                         traditional   Multi-Layer Perceptron           SMOTE   
                         bag-of-words     Logistic Regression     No sampling   
                         bag-of-words  Support Vector Machine  Under-Sampling   
                         bag-of-words           Random Forest  Under-Sampling   
                         bag-of-words                 XGBoost     No sampling   
       all regressions   traditional      Logistic Regression   Over-Sampling   
                         traditional   Support Vector Machine   Over-Sampling   
                         traditional            Random Forest     No sampling   
                         traditional                  XGBoost     No sampling   
                         traditional   Multi-Layer Perceptron     No sampling   
                         bag-of-words     Logistic Regression     No sampling   
                         bag-of-words  Support Vector Machine   Over-Sampling   
                         bag-of-words           Random Forest     No sampling   
                         bag-of-words                 XGBoost     No sampling   
SZZ    perf. regressions traditional      Logistic Regression   Over-Sampling   
                         traditional   Support Vector Machine           SMOTE   
                         traditional            Random Forest   Over-Sampling   
                         traditional                  XGBoost   Over-Sampling   
                         traditional   Multi-Layer Perceptron           SMOTE   
bugbug perf. regressions traditional      Logistic Regression           SMOTE   
                         traditional   Support Vector Machine   Over-Sampling   
                         traditional            Random Forest   Over-Sampling   
                         traditional                  XGBoost     No sampling   
                         traditional   Multi-Layer Perceptron   Over-Sampling   
       all regressions   traditional      Logistic Regression           SMOTE   
                         traditional   Support Vector Machine           SMOTE   
                         traditional            Random Forest     No sampling   
                         traditional                  XGBoost     No sampling   
                         traditional   Multi-Layer Perceptron     No sampling   

                                       AP (val.)  
Data   Target            Feature Type             
SZZ    perf. regressions traditional      0.0304  
                         traditional      0.0365  
                         traditional      0.0528  
                         traditional      0.0590  
                         traditional      0.0375  
                         bag-of-words     0.0670  
                         bag-of-words     0.0563  
                         bag-of-

In [36]:
print(best_sampler.to_latex(index=True))

\begin{tabular}{lllllr}
\toprule
       &                 &             &                   Model &        Sampling &  AP (val.) \\
Data & Target & Feature Type &                         &                 &            \\
\midrule
SZZ & perf. regressions & traditional &     Logistic Regression &     No sampling &     0.0304 \\
       &                 & traditional &  Support Vector Machine &     No sampling &     0.0365 \\
       &                 & traditional &           Random Forest &   Over-Sampling &     0.0528 \\
       &                 & traditional &                 XGBoost &   Over-Sampling &     0.0590 \\
       &                 & traditional &  Multi-Layer Perceptron &     No sampling &     0.0375 \\
       &                 & bag-of-words &     Logistic Regression &     No sampling &     0.0670 \\
       &                 & bag-of-words &  Support Vector Machine &   Over-Sampling &     0.0563 \\
       &                 & bag-of-words &           Random Forest &     No s

/tmp/ipykernel_14058/346280886.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(best_sampler.to_latex(index=True))


In [28]:
best_sampler = []

scoring = 'average_precision'
for config, evaluations in evaluations_by_config.items():
    ws = config.split('_')
    feature_type = ws.pop()
    target = ws.pop()
    data = '_'.join(ws)

    amax = np.argmax([evaluation['test']['regression']['f1'] for evaluation in evaluations])
    evaluation = evaluations[amax]

    sampler = None
    if evaluation['model'] == 'tpot':
        sampler = None
    elif evaluation['best_params']:
        sampler = evaluation['best_params']['sampler']

    best_sampler.append({
        'Data': data,
        'Target': target,
        'Feature Type': feature_type,
        'Model':  model_names[evaluation['model']],
        'recall': evaluation['test']['regression']['recall'],
        'precision': evaluation['test']['regression']['precision'],
        'F1': evaluation['test']['regression']['f1'],
        'Sampling Method': str(sampler)
        })

best_sampler = pd.DataFrame(best_sampler)
best_sampler = best_sampler.replace({
                    'None': 'No sampling',
                    'RandomOverSampler(random_state=0)': 'Random Over-Sampling',
                    'RandomUnderSampler(random_state=0)': 'Random Under-Sampling',
                    'SMOTE(random_state=0)': 'SMOTE'
                    })

best_sampler = best_sampler.replace({
    'fixed_defect_szz': 'SZZ',
    'bugbug_buglevel': 'bugbug'
})
best_sampler = best_sampler.replace({
    'performance': 'perf. regressions',
    'regression': 'all regressions'
})

best_sampler.set_index(pd.MultiIndex.from_frame(best_sampler[['Data', 'Target', 'Feature Type']]), inplace=True)
best_sampler = best_sampler.drop(['Data', 'Target', 'Feature Type'], axis=1)

best_sampler    

Model  recall  \
Data   Target            Feature Type                                   
SZZ    perf. regressions traditional                     TPOT  0.1364   
                         bow           Support Vector Machine  0.1818   
bugbug perf. regressions traditional      Logistic Regression  0.2324   
                         bow              Logistic Regression  0.1535   
       all regressions   traditional            Random Forest  0.4746   
                         bow                          XGBoost  0.4062   

                                       precision     F1       Sampling Method  
Data   Target            Feature Type                                          
SZZ    perf. regressions traditional      0.0326 0.0526           No sampling  
                         bow              0.0288 0.0497  Random Over-Sampling  
bugbug perf. regressions traditional      0.1383 0.1734                 SMOTE  
                         bow              0.2022 0.1745           No sampling  
       all regressions   traditional      0.3943 0.4307           No sampling  
                         bow              0.4168 0.4114           No sampling

In [29]:
print(best_sampler.to_latex(index=True))

\begin{tabular}{llllrrrl}
\toprule
       &                 &     &                   Model &  recall &  precision &     F1 &       Sampling Method \\
Data & Target & Feature Type &                         &         &            &        &                       \\
\midrule
SZZ & perf. regressions & traditional &                    TPOT &  0.1364 &     0.0326 & 0.0526 &           No sampling \\
       &                 & bow &  Support Vector Machine &  0.1818 &     0.0288 & 0.0497 &  Random Over-Sampling \\
bugbug & perf. regressions & traditional &     Logistic Regression &  0.2324 &     0.1383 & 0.1734 &                 SMOTE \\
       &                 & bow &     Logistic Regression &  0.1535 &     0.2022 & 0.1745 &           No sampling \\
       & all regressions & traditional &           Random Forest &  0.4746 &     0.3943 & 0.4307 &           No sampling \\
       &                 & bow &                 XGBoost &  0.4062 &     0.4168 & 0.4114 &           No sampling \\
\bott

/tmp/ipykernel_14058/346280886.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(best_sampler.to_latex(index=True))
